Let's take the code from the previous notebook and add memory to that agent.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

True

# Adding Memory

In [3]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.agents import create_react_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

search = TavilySearchResults()
loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()
retriever.get_relevant_documents("What is langchain?")[0]
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langchain_search",
    "Search for information about LangChain. For any questions about LangChain, you must use this tool!",
)

In [4]:
tools = [search, retriever_tool]

In [5]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:\n\ntavily_search_results_json: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nlangchain_search: Search for information about LangChain. For any questions about LangChain, you must use this tool!\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [tavily_search_results_json, langchain_search]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input q

In [7]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [9]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [10]:
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [11]:
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory)

In [12]:
agent_chain.invoke(input="My name is Lucas and I work as a Machine Learning Engineer")



> Entering new AgentExecutor chain...
Thought: I should use the tools available to gather information about Lucas and his job
Action: tavily_search_results_json
Action Input: "Lucas Machine Learning Engineer"
Observation: [{'url': 'https://www.linkedin.com/pulse/takeaways-from-chapter-3-designing-machine-learning-lucas-qdtme', 'content': 'Published Feb 10, 2024 + Follow Chapter 3: Data Engineering Fundamentals - Chip Huyen Data engineering is a crucial pillar in the development of ML systems, largely due to the importance of...'}, {'url': 'https://learnml.substack.com/p/machine-learning-roadmap-for-2024', 'content': "- 📚Course Highlights:\n- Machine Learning Basics: A beginner-friendly introduction to ML, covering supervised and unsupervised learning.\n- Mathematics for Machine Learning: School Of Machine Learning\nMachine Learning Roadmap For 2024 (With Certifications)\nPlus free courses!\n In our continuous journey to empower and educate, we're thrilled to present a comprehensive r

{'input': 'My name is Lucas and I work as a Machine Learning Engineer',
 'chat_history': '',
 'output': "Lucas is a Machine Learning Engineer who uses LangChain as a framework for developing applications powered by language models. He relies on LangChain's standard interfaces and integrations for modules such as Model I/O, Retrieval, and Agents. He also uses LangSmith, a developer platform, to debug, test, evaluate, and monitor his chains built on any LLM framework."}

In [13]:
agent_chain.invoke(input="What is a similar job to mine?")



> Entering new AgentExecutor chain...
Thought: I should use tavily_search_results_json to find a list of similar job titles.
Action: tavily_search_results_json
Action Input: "Similar job titles to Machine Learning Engineer"
Observation: [{'url': 'https://www.projectpro.io/article/careers-in-ai/961', 'content': "NLP Engineer: Skills And Qualifications\nBelow are some of the skills and educational qualifications you need to become an NLP Engineer-\nTechnical Skills\nProgramming languages (Python, Java, R)\nMachine learning algorithms and frameworks (TensorFlow, PyTorch)\nNatural language processing techniques (tokenization, stemming, lemmatization)\nData analysis/visualization and AI tools\nSoft Skills\nCritical thinking and problem-solving\nStrong analytical and communication skills\nCreativity and innovation\nAbility to work independently and collaboratively\nEducational Qualifications\nBachelor's degree in Computer Science, Linguistics, or related field preferred\nMaster's degree in

{'input': 'What is a similar job to mine?',
 'chat_history': "Human: My name is Lucas and I work as a Machine Learning Engineer\nAI: Lucas is a Machine Learning Engineer who uses LangChain as a framework for developing applications powered by language models. He relies on LangChain's standard interfaces and integrations for modules such as Model I/O, Retrieval, and Agents. He also uses LangSmith, a developer platform, to debug, test, evaluate, and monitor his chains built on any LLM framework.",
 'output': 'Some similar job titles to Machine Learning Engineer include NLP Engineer, Data Scientist, AI Research Scientist, AI Engineer, Robotics Scientist, AI Ethics Officer, AI Quality Assurance Manager, AI Product Manager, and Data Ethicist.'}